# _01_ntd_ridership_utils refactor
06/24/2025

Finally decided to centralize the functions needed to run the monthly/annual ridership reports

initially each report contains duplicate functions, modified to work with their respective dataset. 

Now, all common functions will be refactored to work with both monthly/annual datasets. 

In [1]:
# all imports from _01_utils
import gcsfs
import geopandas as gpd
import os
import pandas as pd
import shutil

from calitp_data_analysis.tables import tbls
from siuba import _, collect, filter, select, distinct
from calitp_data_analysis.sql import to_snakecase
#from segment_speed_utils.project_vars import PUBLIC_GCS

import sys
sys.path.append("./monthly_ridership_report")
from update_vars import GCS_FILE_PATH, NTD_MODES, NTD_TOS, YEAR, MONTH


fs = gcsfs.GCSFileSystem()

RTPA_URL = ("https://services3.arcgis.com/bWPjFyq029ChCGur/arcgis/rest/services/"
       "RTPAs/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"
      )


## Update add_change_columns

In [2]:
def add_change_columns(
    df: pd.DataFrame,
	sort_cols: str,
	group_cols: str,
	change_col: str
) -> pd.DataFrame:
    """
    This function works with the warehouse `dim_monthly_ntd_ridership_with_adjustments` long data format.
    Sorts the df by ntd id, mode, tos, period month and period year. then adds 2 new columns, 1. previous year/month UPT and 2. UPT change 1yr.
    sort_cols and group_cols args needed to specify annual/monthly specific groupings.
    change_col arg is used to name the new column.
    """
    # checks for monthly data specific columns, changes them to int.
    if {"period_year","period_month"}.issubset(df.columns):
        df[["period_year","period_month"]] = df[["period_year","period_month"]].astype(int)

    df = df.assign(
        # unpacks dictionary to use change_col arg
        **{change_col : (df.sort_values(sort_cols)
                        .groupby(group_cols)["upt"] 
                        .apply(lambda x: x.shift(1))
                        )}
    )

    df["change_1yr"] = (df["upt"] - df[change_col])
    
    df = get_percent_change(
        df,
        change_col = change_col
    )
    
    return df

In [ ]:
## moved to function
# for produce_ntd_monthly_ridership_by_rtpa
# in monthly report sript

monthly_sort_cols =  [
    "ntd_id",
    "mode", 
    "tos",
    "period_month", 
    "period_year"
] # got the order correct with ["period_month", "period_year"]! sorted years with grouped months

monthly_group_cols = [
    "ntd_id",
    "mode", 
    "tos"
              ]

monthly_change_col ="previous_y_m_upt"

df = add_change_columns(
    df,
    sort_cols = monthly_sort_cols,
    group_cols = monthly_group_cols,
    change_col = monthly_change_col
)

In [ ]:
## moved to function
# for produce_annual_ntd_ridership_data_by_rtpa
# in annual report script

annual_sort_cols =  [
    "ntd_id",
    "year",
    "mode", 
    "service",
] # got the order correct with ["period_month", "period_year"]! sorted years with grouped months

annual_group_cols = [
    "ntd_id",
    "mode", 
    "service"
              ]

annual_change_col ="previous_y_upt"

df = add_change_columns(
    df,
    sort_cols = annual_sort_cols,
    group_cols = annual_group_cols,
    change_col = annual_change_col
)

## Update get_percent_change

In [3]:
def get_percent_change(
    df: pd.DataFrame,
    change_col: str
) -> pd.DataFrame:
    """
    updated to work with the warehouse `dim_monthly_ntd_ridership_with_adjustments` long data format. 
    
    """
    df["pct_change_1yr"] = (
        (df["upt"] - df[change_col])
        .divide(df["upt"])
        .round(4)
    )
    
    return df

In [ ]:
## added to add_change_columns function
df = get_percent_change(
    df,
    change_col=change_col
)

## update sum_by_group

In [4]:
def sum_by_group(
    df: pd.DataFrame,
    group_cols: list,
    group_col2: list,
    agg_cols: dict,
    change_col: str,
    
) -> pd.DataFrame:
    """
    Since data is now long to begin with, this replaces old sum_by_group, make_long and assemble_long_df functions.
    Separated col groups args so function can be used by annual/monthly report
    """
    grouped_df = df.groupby(group_cols+
                             group_col2
                           ).agg(agg_cols
    ).reset_index()
    
    #get %change back
    grouped_df = get_percent_change(grouped_df, change_col)
    
    #decimal to whole number
    grouped_df["pct_change_1yr"] = grouped_df["pct_change_1yr"]*100
    
    return grouped_df

In [ ]:
# for monthly report notebook
monthly_group_col_2 = [
    'period_year',
    'period_month',
    'period_year_month']

monthly_agg_col = {
        "upt":"sum",
        "previous_y_m_upt":"sum",
        "change_1yr":"sum"
    }

by_,,,_long = sum_by_group(
    df,
    group_cols= ,,,_cols,
    group_col2= monthly_group_col_2,# look into combingin with base grou_cols
    agg_cols = monthly_agg_col
)

In [ ]:
# for annual report notebook 
annual_group_col_2 = ["year"]

monthly_agg_col= {
                "upt": "sum",
                "previous_y_upt": "sum",
                "change_1yr": "sum",
            }

# used in save_rtpa_outputs
by_,,,_long = sum_by_group(
    df,
    group_cols= ,,,_cols,
    group_col2= annual_group_col_2,# look into combingin with base grou_cols
    agg_cols = monthly_agg_col
)

## update save_rtpa_outputs

In [5]:
from typing import Literal


In [6]:
def save_rtpa_outputs(
    df: pd.DataFrame, 
    year: int, 
    month: str,
    report_type: Literal["annual","monthly"],
    cover_sheet_path: str,
    cover_sheet_index_col: str,
    output_file_name: str,
    col_dict: dict = None,
    monthly_upload_to_public: bool = False,
    annual_upload_to_public: bool = False,
):
    """
    Export an excel for each RTPA, adds a READ ME tab, then writes into a folder.
    Zip that folder. 
    Upload zipped file to GCS.
    Updated Args to declare annual/monthly reports.
    """
    
    print("creating individual RTPA excel files")
    
    for i in df["rtpa_name"].unique():
        
        print(f"creating excel file for: {i}")
        
        # Filename should be snakecase
        rtpa_snakecase = i.replace(' ', '_').replace("/","_").lower()
        
        #insertng readme cover sheet, 
        cover_sheet = pd.read_excel(cover_sheet_path, index_col = cover_sheet_index_col)
        cover_sheet.to_excel(
            f"./{year}_{month}/{rtpa_snakecase}.xlsx", sheet_name = "README")

        rtpa_data = (df[df["rtpa_name"] == i]
         .sort_values("ntd_id")
         #got error from excel not recognizing timezone, made list to include dropping "execution_ts" column
         .drop(columns = "_merge")

                   )
        
        if col_dict:
            rtpa_data = rtpa_data.rename(columns=col_dict)
            
        agency_cols = ["ntd_id", "agency", "rtpa_name"]
        mode_cols = ["mode", "rtpa_name"]
        tos_cols = ["tos", "rtpa_name"]
        reporter_type = ["reporter_type", "rtpa_name"]
        
        if report_type == "monthly":
        #column lists for aggregations

            monthly_group_col_2 = [
            'period_year',
            'period_month',
            'period_year_month']

            monthly_agg_col = {
            "upt":"sum",
            "previous_y_m_upt":"sum",
            "change_1yr":"sum"
            }
            monthly_change_col ="previous_y_m_upt"

            by_agency_long = sum_by_group(
                df = rtpa_data,
                group_cols= agency_cols,
                group_col2= monthly_group_col_2,# look into combingin with base grou_cols
                agg_cols = monthly_agg_col,
                change_col= monthly_change_col
            )

            by_mode_long = sum_by_group(
                df = rtpa_data,
                group_cols= mode_cols,
                group_col2= monthly_group_col_2,# look into combingin with base grou_cols
                agg_cols = monthly_agg_col,
                change_col= monthly_change_col
            )
    
            by_tos_long = sum_by_group(
                df = rtpa_data,
                group_cols= tos_cols,
                group_col2= monthly_group_col_2,# look into combingin with base grou_cols
                agg_cols = monthly_agg_col,
                change_col= monthly_change_col
            )
        #writing pages to excel fil
            with pd.ExcelWriter(f"./{year}_{month}/{rtpa_snakecase}.xlsx", mode ="a") as writer:
                rtpa_data.to_excel(writer, sheet_name = "RTPA Ridership Data", index=False)
                by_agency_long.to_excel(writer, sheet_name = "Aggregated by Agency", index=False)
                by_mode_long.to_excel(writer, sheet_name = "Aggregated by Mode", index=False)
                by_tos_long.to_excel(writer, sheet_name = "Aggregated by TOS", index=False)


    
        if report_type == "annual":
            annual_group_col_2 = ["year"]

            annual_agg_col= {
                            "upt": "sum",
                            "previous_y_upt": "sum",
                            "change_1yr": "sum",
                        }
            annual_change_col ="previous_y_upt"
            
            by_agency_long = sum_by_group(
                df = rtpa_data,
                group_cols= agency_cols,
                group_col2= annual_group_col_2,# look into combingin with base grou_cols
                agg_cols = annual_agg_col,
                change_col = annual_change_col
            )

            by_mode_long = sum_by_group(
                df = rtpa_data,
                group_cols= mode_cols,
                group_col2= annual_group_col_2,# look into combingin with base grou_cols
                agg_cols = annual_agg_col,
                change_col = annual_change_col
            )
    
            by_tos_long = sum_by_group(
                df = rtpa_data,
                group_cols= tos_cols,
                group_col2= annual_group_col_2,# look into combingin with base grou_cols
                agg_cols = annual_agg_col,
                change_col = annual_change_col
            )
            by_reporter_type_long = sum_by_group(
                df = rtpa_data,
                group_cols= reporter_type,
                group_col2= annual_group_col_2,# look into combingin with base grou_cols
                agg_cols = annual_agg_col,
                change_col = annual_change_col
            )

  
            #writing pages to excel fil
            with pd.ExcelWriter(f"./{year}_{month}/{rtpa_snakecase}.xlsx", mode ="a") as writer:
                rtpa_data.to_excel(writer, sheet_name = "RTPA Ridership Data", index=False)
                by_agency_long.to_excel(writer, sheet_name = "Aggregated by Agency", index=False)
                by_mode_long.to_excel(writer, sheet_name = "Aggregated by Mode", index=False)
                by_tos_long.to_excel(writer, sheet_name = "Aggregated by TOS", index=False)
                by_reporter_type_long.to_excel(writer, sheet_name="Aggregate by Reporter Type", index=False)
    
    
    print("zipping all excel files")
    
    shutil.make_archive(f"./{output_file_name}", "zip", f"{year}_{month}")
    
    print("Zipped folder")
    
    fs.upload(
        f"./{output_file_name}.zip", 
        f"{GCS_FILE_PATH}{year}_{month}.zip"
    )
    
    if monthly_upload_to_public:
        fs.upload(
            f"./{output_file_name}.zip",
            f"{PUBLIC_GCS}ntd_monthly_ridership/{year}_{month}.zip"
        )
        print("Uploaded to public GCS - monthly report")
        
    if annual_upload_to_public:
        fs.upload(
            f"./{output_file_name}.zip",
            f"{PUBLIC_GCS}ntd_annual_ridership/{year}_{month}_annual_report_data.zip"
        )
    
        print("Uploaded to public GCS - annual report")
    
    print("complete")
    
    return

In [ ]:
# for monthly report script, variables for save_rtpa_output

col_dict ={
    'Uace Cd': "UACE Code",
    'Dt': "Date",
    'Ntd Id': "NTD ID",
    'Tos': "Type of Service",
    'Legacy Ntd Id': "Legacy NTD ID",
    'Upt': "UPT",
    'Vrm': "VRM",
    'Vrh': "VRH",
    'Voms': "VOMS",
    'Rtpa': "RTPA",
    'Previous Y M Upt': "Previous Year/Month UPT",
    'Change 1Yr': "Change in 1 Year UPT",
    'Pct Change 1Yr': "Percent Change in 1 Year UPT",
    'Tos Full': "Type of Service Full Name"
}

monthly_cover_sheet_path = "./cover_sheet_template.xlsx"
monthly_index_col = "**NTD Monthly Ridership by RTPA**"
monthly_data_file_name = f"{year}_{month}_monthly_report_data"

In [ ]:
# for annual report sript, variables for save_rtpa_output
col_dict= {
    "source_agency":"agency",
    "service":"tos"
}

annual_cover_sheet_path = "./annual_report_cover_sheet_template.xlsx"
annual_index_col = "**NTD Annual Ridership by RTPA**"
annual_data_file_name = f"{year}_{month}_annual_report_data"

## update remove_local_outputs

In [7]:
def remove_local_outputs(
    year: int, 
    month: str
):
    """
    Removes YEAR_MONTH folder and the individual RTPA excel sheets, and
    deletes the YEAR_MONTH zip file from the save_rtpa_outputs function.
    """
    try:
        print("removing data folder")
        shutil.rmtree(f"{year}_{month}/")
    except FileNotFoundError:
        print ("data folder not found")
    
    if os.path.exists(f"{year}_{month}_annual_report_data.zip"):
        os.remove(f"{year}_{month}_annual_report_data.zip")
        print("removing annual data zip file")
    
    elif os.path.exists(f"{year}_{month}_monthly_report_data.zip"):
        os.remove(f"{year}_{month}_monthly_report_data.zip")
        print("removing monthly data zip file")
    
    else:
        print("Could not find report data to delete")
    
    

In [ ]:
# for monthly ridership script


In [ ]:
# for annual ridership script


## update produce_ntd_monthly_ridership_by_rtpa

In [8]:
def produce_ntd_monthly_ridership_by_rtpa(year: int, month: int) -> pd.DataFrame:
    """
    This function works with the warehouse `dim_monthly_ntd_ridership_with_adjustments` long data format.
    Import NTD data from warehouse, filter to CA,
    merge in crosswalk, checks for unmerged rows, then creates new columns for full Mode and TOS name.

    """

    full_upt = (
        tbls.mart_ntd.dim_monthly_ridership_with_adjustments()
        >> filter(
            _.period_year.isin(
                ["2018", "2019", "2020", "2021", "2022", "2023", "2024", "2025"]
            )
        )
        >>select(
            _.ntd_id,
            _.agency,
            _.reporter_type,
            _.period_year_month,
            _.period_year,
            _.period_month,
            _.mode,
            _.tos,
            _.mode_type_of_service_status,
            _.primary_uza_name,
            _.upt
            
        )
        >> collect()
    ).rename(
        columns={
            "mode_type_of_service_status": "Status",
            "primary_uza_name": "uza_name",
        }
    )

    full_upt = full_upt[full_upt.agency.notna()].reset_index(drop=True)

    # full_upt.to_parquet(
    #     f"{GCS_FILE_PATH}ntd_monthly_ridership_{year}_{month}.parquet"
    # )

    ca = full_upt[
        (full_upt["uza_name"].str.contains(", CA")) & (full_upt.agency.notna())
    ].reset_index(drop=True)

    # use new crosswalk function
    crosswalk = ntd_id_to_rtpa_crosswalk(split_scag=True)

    min_year = 2018

    # get agencies with last report year and data after > 2018.
    last_report_year = (
        tbls.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_upt()
        >> filter(
            _.year >= min_year,  # see if this changes anything
            _.last_report_year >= min_year,
            _.primary_uza_name.str.contains(", CA")
            | _.primary_uza_name.str.contains("CA-NV")
            | _.primary_uza_name.str.contains("California Non-UZA"),
        )
        >> distinct(
            "source_agency",
            #'agency_status',
            #'legacy_ntd_id',
            "last_report_year",
            #'mode',
            "ntd_id",
            #'reporter_type',
            #'reporting_module',
            #'service',
            #'uace_code',
            #'primary_uza_name',
            #'uza_population',
            #'year',
            #'upt',
        )
        >> collect()
    )

    # merge last report year to CA UPT data
    df = pd.merge(ca, last_report_year, left_on="ntd_id", right_on="ntd_id", how="inner")

    # merge crosswalk to CA last report year
    df = pd.merge(
        df,
        # Merging on too many columns can create problems
        # because csvs and dtypes aren't stable / consistent
        # for NTD ID, Legacy NTD ID, and UZA
        crosswalk[["ntd_id_2022", "rtpa_name"]],
        left_on="ntd_id",
        right_on="ntd_id_2022",
        how="left",
        indicator=True,
    )

    print(df._merge.value_counts())

    # check for unmerged rows
    if len(df[df._merge == "left_only"]) > 0:
        raise ValueError("There are unmerged rows to crosswalk")
    
    monthly_sort_cols =  [
    "ntd_id",
    "mode", 
    "tos",
    "period_month", 
    "period_year"
] # got the order correct with ["period_month", "period_year"]! sorted years with grouped months

    monthly_group_cols = [
        "ntd_id",
        "mode", 
        "tos"
                  ]

    monthly_change_col ="previous_y_m_upt"

    df = add_change_columns(
        df,
        sort_cols = monthly_sort_cols,
        group_cols = monthly_group_cols,
        change_col = monthly_change_col
    )

    
    df = df.assign(
        Mode_full = df["mode"].map(NTD_MODES),
        TOS_full = df["tos"].map(NTD_TOS)
    )
    
    return df

## update produce_annual_ntd_ridership_data_by_rtpa

In [9]:
def produce_annual_ntd_ridership_data_by_rtpa(min_year: str, split_scag: bool) -> pd.DataFrame:
    """
    Function that ingest time series ridership data from `mart_ntd_funding_and_expenses.fct_service..._by_mode_upt`. 
    Filters for CA agencies with last report year and year of data greater than min_year
    Merges in ntd_id_to_rtpa_crosswalk function. Aggregates by agency, mode and TOS. calculates change in UPT.
    """
    #from annual_ridership_module import add_change_columns
    
    
    print("ingest annual ridership data from warehouse")
    
    ntd_service =(
        tbls.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_upt()
        >> filter(
            _.year >= min_year,
            _.last_report_year >= min_year,
            _.primary_uza_name.str.contains(", CA") | 
            _.primary_uza_name.str.contains("CA-NV") |
            _.primary_uza_name.str.contains("California Non-UZA") 
        )
        >> select(
            'source_agency',
            'agency_status',
            'legacy_ntd_id',
            'last_report_year',
            'mode',
            'ntd_id',
            'reporter_type',
            'reporting_module',
            'service',
            'uace_code',
            'primary_uza_name',
            'uza_population',
            'year',
            'upt',
        )
        >> collect())
    
    ntd_service = (
        ntd_service.groupby(
            [
                "source_agency",
                "agency_status",
                #"city",
                #"state",
                "ntd_id",
                "primary_uza_name",
                "reporter_type",
                "mode",
                "service",
                "last_report_year",
                "year",
            ]
        )
        .agg({"upt": "sum"})
        .sort_values(by="ntd_id")
        .reset_index()
    )
    
    print("create crosswalk from ntd_id_to_rtpa_crosswalk function")
    
    # Creating crosswalk using function, enable splitting scag to indivdual CTC
    ntd_to_rtpa_crosswalk = ntd_id_to_rtpa_crosswalk(split_scag=split_scag)
    
    
    print("merge ntd data to crosswalk")
    # merge service data to crosswalk
    ntd_data_by_rtpa = ntd_service.merge(
        ntd_to_rtpa_crosswalk,
        how="left",
        left_on=[
            "ntd_id",
            # "agency", "reporter_type", "city" # sometime agency name, reporter type and city name change or are inconsistent, causing possible fanout
        ],
        right_on="ntd_id_2022",
        indicator=True,
    )#.rename(columns={"source_agency":"agency"})
    
    # list of ntd_id with LA County Dept of Public Works name
    lacdpw_list = [
        "90269",
        "90270",
        "90272",
        "90273",
        "90274",
        "90275",
        "90276",
        "90277",
        "90278",
        "90279",
    ]
    
    # replace LA County Public Works agencies with their own RTPA
    ntd_data_by_rtpa.loc[
        ntd_data_by_rtpa["ntd_id"].isin(lacdpw_list), ["rtpa_name", "_merge"]
    ] = ["Los Angeles County Department of Public Works", "both"]
    
    print(ntd_data_by_rtpa._merge.value_counts())
        
    if len(ntd_data_by_rtpa[ntd_data_by_rtpa._merge=="left_only"]) > 0:
        raise ValueError("There are unmerged rows to crosswalk")
    
    print("add `change_column` to data")
    
    annual_sort_cols =  [
        "ntd_id",
        "year",
        "mode", 
        "service",
    ] # got the order correct with ["period_month", "period_year"]! sorted years with grouped months

    annual_group_cols = [
        "ntd_id",
        "mode", 
        "service"
                  ]

    annual_change_col ="previous_y_upt"

    ntd_data_by_rtpa = add_change_columns(
        ntd_data_by_rtpa,
        sort_cols = annual_sort_cols,
        group_cols = annual_group_cols,
        change_col = annual_change_col
    )
    
    print("map mode and tos desc.")
    ntd_data_by_rtpa = ntd_data_by_rtpa.assign(
        mode_full = ntd_data_by_rtpa["mode"].map(NTD_MODES),
        service_full = ntd_data_by_rtpa["service"].map(NTD_TOS)
    )
    print("complete")
    return ntd_data_by_rtpa

# TEST

In [10]:
from _01_ntd_ridership_utils import ntd_id_to_rtpa_crosswalk

In [ ]:
# monthly_ridership_by_rtpa script

In [11]:
monthly = produce_ntd_monthly_ridership_by_rtpa(YEAR, MONTH)

monthly.columns

both          28864
left_only         0
right_only        0
Name: _merge, dtype: int64


/tmp/ipykernel_394/3131337192.py:18: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: x.shift(1))


Index(['ntd_id', 'agency', 'reporter_type', 'period_year_month', 'period_year',
       'period_month', 'mode', 'tos', 'Status', 'uza_name', 'upt',
       'source_agency', 'last_report_year', 'ntd_id_2022', 'rtpa_name',
       '_merge', 'previous_y_m_upt', 'change_1yr', 'pct_change_1yr',
       'Mode_full', 'TOS_full'],
      dtype='object')

In [12]:
# annua_ridership_module script
annual = produce_annual_ntd_ridership_data_by_rtpa(2018, split_scag=True)

annual.columns

ingest annual ridership data from warehouse
create crosswalk from ntd_id_to_rtpa_crosswalk function
merge ntd data to crosswalk
both          3768
left_only        0
right_only       0
Name: _merge, dtype: int64
add `change_column` to data
map mode and tos desc.
complete


/tmp/ipykernel_394/3131337192.py:18: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: x.shift(1))


Index(['source_agency', 'agency_status', 'ntd_id', 'primary_uza_name',
       'reporter_type', 'mode', 'service', 'last_report_year', 'year', 'upt',
       'name', 'ntd_id_2022', 'rtpa_name', 'mpo_name', 'key',
       'county_geography_name', 'organization_key', '_merge', 'previous_y_upt',
       'change_1yr', 'pct_change_1yr', 'mode_full', 'service_full'],
      dtype='object')

In [ ]:
# test making output folder, saving output excels, removing 

In [17]:
os.makedirs(f"./{YEAR}_{MONTH}/")

In [18]:
#monthly script test

monthly_col_dict ={
    'Uace Cd': "UACE Code",
    'Dt': "Date",
    #'Ntd Id': "NTD ID",
    'Tos': "Type of Service",
    'Legacy Ntd Id': "Legacy NTD ID",
    #'Upt': "UPT",
    'Vrm': "VRM",
    'Vrh': "VRH",
    'Voms': "VOMS",
    'Rtpa': "RTPA",
    #'Previous Y M Upt': "Previous Year/Month UPT",
    #'Change 1Yr': "Change in 1 Year UPT",
    'Pct Change 1Yr': "Percent Change in 1 Year UPT",
    'Tos Full': "Type of Service Full Name"
}

monthly_cover_sheet_path = "monthly_ridership_report/cover_sheet_template.xlsx"
monthly_index_col = "**NTD Monthly Ridership by RTPA**"
monthly_data_file_name = f"{YEAR}_{MONTH}_monthly_report_data"

save_rtpa_outputs(
    df=monthly, 
    year = YEAR, 
    month = MONTH,
    #col_dict = monthly_col_dict,
    cover_sheet_path = monthly_cover_sheet_path,
    cover_sheet_index_col = monthly_index_col,
    output_file_name = monthly_data_file_name,
    report_type = "monthly"
)

creating individual RTPA excel files
creating excel file for: Metropolitan Transportation Commission
creating excel file for: Los Angeles County Metropolitan Transportation Authority
creating excel file for: Santa Cruz County Regional Transportation Commission
creating excel file for: San Joaquin Council of Governments
creating excel file for: Santa Barbara County Association of Governments
creating excel file for: San Diego Association of Governments
creating excel file for: Riverside County Transportation Commission
creating excel file for: Orange County Transportation Authority
creating excel file for: Sacramento Area Council of Governments
creating excel file for: Transportation Agency for Monterey County
creating excel file for: Tulare County Association of Governments
creating excel file for: San Bernardino County Transportation Authority
creating excel file for: Ventura County Transportation Commission
creating excel file for: Placer County Transportation Planning Agency
creatin

In [19]:
remove_local_outputs(YEAR, MONTH)

removing data folder
removing monthly data zip file


In [14]:
# annual script test
annual_col_dict= {
    "source_agency":"agency",
    "service":"tos"
}

annual_cover_sheet_path = "annual_ridership_report/annual_report_cover_sheet_template.xlsx"
annual_index_col = "**NTD Annual Ridership by RTPA**"
annual_data_file_name = f"{YEAR}_{MONTH}_annual_report_data"

save_rtpa_outputs(
    df=annual, 
    year = YEAR, 
    month = MONTH,
    col_dict = annual_col_dict,
    cover_sheet_path = annual_cover_sheet_path,
    cover_sheet_index_col = annual_index_col,
    output_file_name = annual_data_file_name,
    report_type = "annual"
)

creating individual RTPA excel files
creating excel file for: Metropolitan Transportation Commission
creating excel file for: Kern Council of Governments
creating excel file for: Santa Cruz County Regional Transportation Commission
creating excel file for: Stanislaus Council of Governments
creating excel file for: Los Angeles County Metropolitan Transportation Authority
creating excel file for: San Joaquin Council of Governments
creating excel file for: Sacramento Area Council of Governments
creating excel file for: Santa Barbara County Association of Governments
creating excel file for: San Diego Association of Governments
creating excel file for: Fresno Council of Governments
creating excel file for: San Bernardino County Transportation Authority
creating excel file for: Riverside County Transportation Commission
creating excel file for: Ventura County Transportation Commission
creating excel file for: Orange County Transportation Authority
creating excel file for: Transportation Age

In [15]:
remove_local_outputs(YEAR, MONTH)

removing data folder
removing annual data zip file


In [23]:
read_annual = pd.read_parquet(f"{GCS_FILE_PATH}annual_ridership_report_data.parquet")

read_annual.columns.tolist()

['source_agency',
 'agency_status',
 'ntd_id',
 'primary_uza_name',
 'reporter_type',
 'mode',
 'service',
 'last_report_year',
 'year',
 'upt',
 'name',
 'ntd_id_2022',
 'rtpa_name',
 'mpo_name',
 'key',
 'county_geography_name',
 'organization_key',
 '_merge',
 'previous_y_upt',
 'change_1yr',
 'pct_change_1yr',
 'mode_full',
 'service_full']